In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import datetime
import dateutil.parser
from os.path import join

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    get_visualization_subtitle,
    get_country_color_map,
    apply_theme,
)
from web import for_website

alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

# Data Preprocessing

## Site Descriptions From Figshare

In [ ]:
DATA_RELEASE = "2020-11-06"

In [ ]:
sites_df = pd.read_csv(join("..", "data", "Health_Systems_Participating.tsv"), sep='\t', skiprows=2, header=None, thousands=',')
sites_column_map = {
    0: "site_name",
    1: "siteid",
    2: "city",
    3: "country",
    4: "patient_type",
    6: "adult_num_hosp",
    7: "adult_num_beds",
    8: "adult_num_yearly_discharge",
    10: "ped_num_hosp",
    11: "ped_num_beds",
    12: "ped_num_yearly_discharge",
    32: "lat",
    33: "lon"
}
sites_df = sites_df.rename(columns=sites_column_map)
sites_df = sites_df[list(sites_column_map.values())]
sites_df = sites_df.dropna(subset=["site_name"])
sites_df = sites_df.dropna(subset=["lat", "lon"])
sites_df = sites_df.set_index("siteid")
sites_df["country"] = sites_df["country"].replace({"UK": "United Kingdom"})
sites_df.head()

In [ ]:
num_sites_na = sites_df.loc[sites_df["country"] == "USA"].shape[0]
num_sites_eur = sites_df.loc[sites_df["country"].isin(["Spain", "United Kingdom", "Italy", "France", "Germany"])].shape[0]
num_sites_asia = sites_df.loc[sites_df["country"].isin(["Singapore"])].shape[0]
num_sites_sa = sites_df.loc[sites_df["country"].isin(["Brazil"])].shape[0]

## Topology of Countries

In [ ]:
topo_url = "https://raw.githubusercontent.com/danutzthe/topojson_world_map/master/world-110m.json"
countries = alt.topo_feature(topo_url, 'countries')

## Country Codes

In [ ]:
code_df = pd.read_json(
    "https://raw.githubusercontent.com/alisle/world-110m-country-codes/master/world-110m-country-codes.json"
)

# Add a missing country
singapore_df = pd.DataFrame(data=[{"code": "Singapore", "id": 702, "name": "Singapore"}])
code_df = pd.concat([code_df, singapore_df])

## Merge Data

In [ ]:
# Add custom encodigs to code_df
df = pd.DataFrame(data=[
    # Custom encoding for each city
    {"id": 250, "color": "#0072B2", "fill": "lightgray", "opacity": 1, "country": "France"},
    {"id": 276, "color": "#E69F00", "fill": "lightgray", "opacity": 1, "country": "Germany"},
    {"id": 380, "color": "#009E73", "fill": "lightgray", "opacity": 1, "country": "Italy"},
    {"id": 702, "color": "#CC79A7", "fill": "lightgray", "opacity": 1, "country": "Singapore"},
    {"id": 840, "color": "#D55E00", "fill": "lightgray", "opacity": 1, "country": "USA"},
    {"id": 826, "color": "#382633", "fill": "lightgray", "opacity": 1, "country": "United Kingdom"},
    {"id": 724, "color": "#A3333D", "fill": "lightgray", "opacity": 1, "country": "Spain"},
    {"id": 76, "color": "#0E3B43", "fill": "lightgray", "opacity": 1, "country": "Brazil"},
])
code_df = code_df.set_index("id")
df = df.set_index("id")
df = code_df.join(df).reset_index()
df.loc[df["opacity"] != 1, "opacity"] = 0.2 # default opacity

# Add site descriptions
df = df.set_index("country")
sites_df = sites_df.set_index("country")
df = df.join(sites_df).reset_index()

# Add lists of sites for each city
UNIQUE_CITIES = sites_df["city"].unique().tolist()
for c in UNIQUE_CITIES:
    f = df["city"] == c
    sites_str =  ", ".join(sites_df[sites_df["city"] == c]["site_name"].unique().tolist())
    df.loc[f, "sites"] = sites_str

# Visualization: Symbol Map

## Map Background

In [ ]:
background = alt.Chart(countries).mark_geoshape(
    fill="lightgray",
    stroke='lightgray'
).encode(
    opacity=alt.Opacity("opacity:Q", legend=None)
).transform_lookup(
    lookup="id",
    from_=alt.LookupData(df, "id", ["opacity", "color"])
)

In [ ]:
df = df.rename(columns={'country': 'Country', 'sites': 'Sites', 'city': 'City', 'adult_num_hosp': 'Hospitals', 'lon': 'Longitude', 'lat': 'Latitude'})

## Symbols

In [ ]:
COUNTRIES = ["France", "Germany", "Italy", "Singapore", "USA", "United Kingdom", "Spain", "Brazil"]
COUNTRY_COLOR = ["#0072B2", "#E69F00", "#009E73", "#CC79A7", "#D55E00", "#382633", "#A3333D", "#0E3B43"]
color_scale = alt.Scale(domain=COUNTRIES, range=COUNTRY_COLOR)

circle = alt.Chart(df).transform_aggregate(
    Latitude_Mean='mean(Latitude)',
    Longitude_Mean='mean(Longitude)',
    Hospitals_Total='sum(Hospitals)',
    Sites_Total='count()',
    groupby=['City','Country', 'Sites']
).transform_filter(
    alt.datum["City"] != None
).mark_circle().encode(
    longitude='Longitude_Mean:Q',
    latitude='Latitude_Mean:Q',
    size=alt.Size(
        'Hospitals_Total:Q', 
        title='Number of Hospitals', 
        scale=alt.Scale(domain=[1, 40], range=[60, 400]),
        legend=None
    ),
    color=alt.Color(
        'Country:N', 
        scale=color_scale, 
        legend=None
    ),
    tooltip=[
        alt.Tooltip('Country:N'),
        alt.Tooltip('City:N'),
        alt.Tooltip('Sites:N', title="Participating sites"),
        alt.Tooltip('Sites_Total:Q', title="Number of sites"), 
        alt.Tooltip('Hospitals_Total:Q', title="Number of hospitals"),
    ]
)

labels = circle.mark_text(
    align='center',
    baseline='top',
    dy=7
).encode(
    longitude='Longitude_Mean:Q',
    latitude='Latitude_Mean:Q',
    text='City',
    size=alt.value(8),
    color=alt.value('black')
)

## Map Visualizations by Continent

In [ ]:
nor_am = ( background + circle + labels ).project(
    type= 'mercator',
    scale= 280,
    center= [-126, 59],
    translate = [0, 0]
).properties(
    title={
        "text": 'Sites in North America',
        "subtitle": get_visualization_subtitle(num_sites=num_sites_na, data_release=DATA_RELEASE),
        "subtitleColor": "gray"
    },
    width=300, height=300
)

nor_am

In [ ]:
sou_am = ( background + circle + labels ).project(
    type= 'mercator',
    scale= 180,
    center= [-110, 25],
    translate = [0, 0]
).properties(
    title={
        "text": 'Sites in South America',
        "subtitle": get_visualization_subtitle(num_sites=num_sites_sa, data_release=DATA_RELEASE),
        "subtitleColor": "gray"
    },
    width=300, height=300
)

sou_am

In [ ]:
label_offsets = {
    # Custom dx and dy
    "Paris": {"dx": 0, "dy": 16},
    "Milano": {"dx": -20, "dy": 0},
    "Bergamo": {"dx": 0, "dy": -10},
    "Erlangen": {"dx": 8, "dy": 10},
}

labels_with_offset = labels.transform_filter(
    (alt.datum["City"] != "Paris") & 
    (alt.datum["City"] != "Milano") & 
    (alt.datum["City"] != "Bergamo") & 
    (alt.datum["City"] != "Erlangen")
)

for city in list(label_offsets.keys()):
    labels_with_offset += labels.transform_filter(
        alt.datum["City"] == city
    ).mark_text(dx=label_offsets[city]["dx"], dy=label_offsets[city]["dy"])

eu = ( background + circle + labels_with_offset ).project(
    type='mercator',
    scale=420,
    center=[-20, 58],
    translate=[0, 0],
).properties(
    title={
        "text": 'Sites in Europe',
        "subtitle": get_visualization_subtitle(num_sites=num_sites_eur, data_release=DATA_RELEASE),
        "subtitleColor": "gray"
    },
    width=300, height=300
)

eu

In [ ]:
asia = ( background + circle + labels ).project(
    type= 'mercator',
    scale= 900,
    center= [94,10],
    translate=[0, 0]
).properties(
    title={
        "text": 'Sites in Asia',
        "subtitle": get_visualization_subtitle(num_sites=num_sites_asia, data_release=DATA_RELEASE),
        "subtitleColor": "gray"
    },
    width=300, height=300
)

asia

In [ ]:
h = apply_theme(alt.hconcat(alt.vconcat(nor_am, sou_am, spacing=10), alt.vconcat(eu, asia, spacing=10), spacing=10)).properties(
    background="transparent"
)

for_website(h, "Map", "Sites by continent horizontal")

h

In [ ]:
v = apply_theme(alt.vconcat(nor_am, eu, asia, sou_am, spacing=10)).properties(
    background="transparent"
)

for_website(v, "Map", "Sites by continent vertical")

v